# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from copy import deepcopy

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.non_reactive_stage import non_reactive_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Construct a single reboiler

In [6]:
model.reboiler = pe.Block(rule=non_reactive_stage_rule)

> Importing Non Reactive Stage......
> Adding the following local variable:
------------------------------------
| reboiler.T_F
| reboiler.P
| reboiler.Q_main
| reboiler.x_
| reboiler.y_
| reboiler.x
| reboiler.y
| reboiler.z
| reboiler.L
| reboiler.V
| reboiler.F
| reboiler.H_L_
| reboiler.H_V_
| reboiler.H_L
| reboiler.H_V
| reboiler.T
| reboiler.H_F
| reboiler.f_V
| reboiler.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.energy_block.dH_F
| reboiler.energy_block.dH_V
| reboiler.energy_block.dH_L
| reboiler.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.VLE_block.P_VLE
| reboiler.VLE_block.n_ave
| reboiler.VLE_block.n_ave_cal
| reboiler.VLE_block.Hen
| reboiler.VLE_block.Hen0
| reboiler.VLE_block.gamma
| r

# Linking Stage Variables

### Vapor Between Reactive Stages

In [7]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [8]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser Vapor

In [9]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

### Condenser Liquid

In [10]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

### Reboiler Vapor

In [11]:
def V_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].V['in'] == model.reboiler.V['out']
model.V_reboiler_con = pe.Constraint(rule=V_reboiler_rule)

def Vy_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].y_['in',i] == model.reboiler.y[i]
model.Vy_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_reboiler_rule)

def Vh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_V_['in'] == model.reboiler.H_V
model.Vh_reboiler_con = pe.Constraint(rule=Vh_reboiler_rule)

### Reboiler Liquid

In [12]:
def L_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].L['out'] == model.reboiler.L['in']
model.L_reboiler_con = pe.Constraint(rule=L_reboiler_rule)

def Lx_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].x[i] == model.reboiler.x_['in',i]
model.Lx_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_reboiler_rule)

def Lh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_L == model.reboiler.H_L_['in']
model.Lh_reboiler_con = pe.Constraint(rule=Lh_reboiler_rule)

In [13]:
# model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY)\
#                          - model.reboiler.MPCC.pf ,sense=pe.maximize)
model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
# model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load from Reactive Flash solutions

In [15]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    results_imported = pickle.load(f)
results_changed = deepcopy(results_imported)

### Duplicate variable solution and bounds multiplier

In [16]:
results_changed.Solution.Variable = {}
for i in list(results_imported.Solution.Variable.keys()):
    for j in model.TRAY:
        results_changed.Solution.Variable['reactive[{}].{}'.format(j,i)] = \
        results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [17]:
results_changed.Solution.Constraint = {}
for i in list(results_imported.Solution.Constraint.keys()):
    for j in model.TRAY:
        results_changed.Solution.Constraint['reactive[{}].{}'.format(j,i)] = \
        results_imported.Solution.Constraint[i]

# Load from Condenser solutions

In [18]:
with open('../saved_solutions/condenser_MPCC_P_pf_30C.pickle', 'rb') as f:
    condenser_results_imported = pickle.load(f)

### Duplicate variable solution and bounds multiplier

In [19]:
for i in list(condenser_results_imported.Solution.Variable.keys()):
    results_changed.Solution.Variable['condenser.{}'.format(i)] = \
    condenser_results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [20]:
for i in list(condenser_results_imported.Solution.Constraint.keys()):
    results_changed.Solution.Constraint['condenser.{}'.format(i)] = \
    condenser_results_imported.Solution.Constraint[i]

# Load from Reboiler solutions

In [21]:
with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle', 'rb') as f:
    reboiler_results_imported = pickle.load(f)

### Duplicate variable solution and bounds multiplier

In [22]:
for i in list(reboiler_results_imported.Solution.Variable.keys()):
    results_changed.Solution.Variable['reboiler.{}'.format(i)] = \
    reboiler_results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [23]:
for i in list(reboiler_results_imported.Solution.Constraint.keys()):
    results_changed.Solution.Constraint['reboiler.{}'.format(i)] = \
    reboiler_results_imported.Solution.Constraint[i]

In [24]:
model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [25]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [26]:
# reboiler
model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.F.fix(0)
model.reboiler.T_F.fix(300+273.15)
model.reboiler.z.fix(0)

model.reboiler.V['P'].fix(0)
model.reboiler.V['in'].fix(0)
for i in m.COMP_TOTAL: model.reboiler.y_['in',i].fix(0)
model.reboiler.H_V_['in'].fix(0)
model.reboiler.L['P'].fix(0)

# Load Operating Parameters

In [27]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reboiler
model.reboiler.P.fix(20)
# model.reboiler.T.fix(30+273.15)
# model.reboiler.T.fix(350+273.15)
model.reboiler.Q_main.fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0/2)
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(200+273.15)

In [28]:
check_DOF(pe,model)

Active Equality Constraints:	 3420
Active Inequality Constraints:	 3
Active Variables:		 3618
Fixed Variables:		 192
DOF:				 6


In [29]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'

opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                                 mu_init = 1e-06                 yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
              warm_start_mult_bound_push = 1e-20                 yes

****************************************************

  69r-4.7272661e+02 8.74e+01 9.99e+02   1.3 0.00e+00    -  0.00e+00 2.51e-08R  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70r-4.7272661e+02 8.74e+01 3.00e+03   1.3 2.63e+03    -  1.86e-03 4.21e-08f  2
  71r-4.7270917e+02 7.67e+01 4.54e+03  -0.1 3.00e+03    -  8.68e-03 2.82e-03f  1
  72 -4.7270880e+02 7.67e+01 1.39e+04  -6.0 4.86e+04    -  2.24e-03 5.22e-05h  1
  73 -4.7270882e+02 7.67e+01 6.40e+04  -6.0 5.62e+06    -  4.59e-06 1.02e-06h  1
  74 -4.7270883e+02 7.67e+01 5.01e+05  -6.0 7.77e+06    -  2.09e-05 6.51e-07h  4
  75 -4.7270941e+02 7.67e+01 2.72e+05  -6.0 9.90e+06    -  4.10e-06 1.99e-05h  1
  76 -4.7270915e+02 7.67e+01 2.68e+05  -6.0 2.42e+07    -  3.97e-06 3.97e-06s  4
  77r-4.7270915e+02 7.67e+01 1.00e+03   1.1 0.00e+00    -  0.00e+00 0.00e+00R  1
  78r-4.7270915e+02 7.67e+01 1.87e+04   1.1 1.28e+03    -  5.79e-05 4.40e-08f  1
  79r-4.7270519e+02 7.63e+01 1.86e+04   1.1 1.26e+03    -  5.27e-03 1.29e-04f  1
iter    objective    inf_pr 

 162r-4.7246789e+02 8.69e+00 9.99e+02  -1.0 0.00e+00    -  0.00e+00 3.97e-07R  2
 163r-4.7246789e+02 8.69e+00 1.00e+03  -1.0 2.95e+02    -  1.03e-03 9.81e-08f  2
 164r-4.7246758e+02 8.78e+00 9.96e+02  -1.0 2.82e+02    -  4.97e-03 2.03e-03f  1
 165 -4.7246842e+02 8.76e+00 3.78e+03  -6.0 4.74e+03    -  2.62e-02 2.43e-03h  1
 166r-4.7246842e+02 8.76e+00 9.99e+02  -1.3 0.00e+00    -  0.00e+00 1.13e-09R  2
 167r-4.7246857e+02 8.76e+00 9.99e+02  -1.3 3.09e+03    -  2.12e-05 3.87e-06f  1
 168r-4.7246839e+02 8.76e+00 9.99e+02  -1.3 1.84e+02    -  2.37e-05 1.09e-04f  1
 169r-4.7246838e+02 8.76e+00 9.99e+02  -1.3 1.74e+02    -  8.17e-03 1.45e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170r-4.7246733e+02 8.88e+00 9.88e+02  -1.3 1.53e+02    -  5.10e-03 1.09e-02f  1
 171 -4.7246738e+02 8.88e+00 2.08e+02  -6.0 1.76e+03    -  3.09e-02 1.48e-04h  1
 172 -4.7246766e+02 8.87e+00 4.17e+03  -6.0 4.74e+03    -  2.74e-02 8.26e-04h  1
 173 -4.7246769e+02 8.87e+00

 261 -4.4727913e+02 6.68e-01 4.28e+04  -6.0 6.95e+02    -  2.78e-03 5.23e-02f  3
 262 -4.4763129e+02 6.36e-01 5.57e+04  -6.0 5.68e+02    -  1.36e-01 4.85e-02f  3
 263 -4.4772068e+02 6.28e-01 5.54e+04  -6.0 4.50e+02    -  1.70e-01 1.20e-02f  5
 264 -4.4791889e+02 6.13e-01 5.64e+04  -6.0 4.19e+02    -  2.19e-01 2.51e-02f  4
 265 -4.4883924e+02 5.46e-01 1.09e+05  -6.0 3.58e+02    -  2.80e-01 1.11e-01f  2
 266 -4.5030847e+02 4.50e-01 1.53e+05  -6.0 1.55e+02    -  3.74e-01 1.82e-01f  1
 267 -4.5030847e+02 4.50e-01 1.53e+05  -6.0 2.93e+03    -  6.31e-06 5.91e-08h  3
 268 -4.5041751e+02 4.44e-01 1.52e+05  -6.0 1.05e+02    -  1.73e-08 1.38e-02f  1
 269 -4.5136945e+02 3.91e-01 1.85e+05  -6.0 1.05e+02    -  4.90e-02 1.22e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270 -4.5240691e+02 3.33e-01 1.64e+05  -6.0 8.49e+01    -  2.87e-01 1.51e-01f  1
In iteration 270, 1 Slack too small, adjusting variable bound
 271 -4.5240692e+02 3.33e-01 1.64e+05  -6.0 4.7

In [30]:
model.reactive[1].VLE_block.P_VLE.value

20.000000234582448

In [31]:
model.reactive[1].V['out'].value

0.477671914266457

In [32]:
beautify(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
T			 Q			 V			 L
30.0 	 -12.34255202320544 	 0.2644427454961411 	 0
200.0 	 -43.399062143066914 	 0.477671914266457 	 0.009394949005439988
200.00000363759392 	 0 	 0.0 	 0.009394950765703155
----------------------------------------------------------------------------------------------------
Top Product V/L/W
0.2644427454961411
0.004922239447260747
0.19845256263075048
----------------------------------------------------------------------------------------------------
Bottom Product L
0.009394950765703155
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Reboiler	Vapor		Liquid
H2 		70.171%		0.924%				38.857%		1.358%
CO 		8.922%		0.252%				4.942%		0.231%
CO2 		18.724%		3.638%				10.403%		1.216%
H2O 		0.223%		0.000%				43.732%		12.701%
C2H4 		0.258%		0.081%				0.144%		0.021%
C3H6 		0.290%		0.

In [33]:
model.reactive[1].kinetics_block.r_FT_total.value

0.25989787956614424

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/3_stage_condenser_240C.pickle','wb') as f:
#     pickle.dump(results,f)

# So, what exactly does adding a reflux do?

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [ ]:
Refluxrange = np.linspace(0,2,41)
Trange = np.linspace(300+273.15,200+273.15,21)

In [ ]:
cd_data_master = {}
rf_data_master = {}

for t in Trange:
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
    model.solutions.load_from(results_anchor)
    update_dual(pe,model)
    
    for j in model.reactive:
        model.reactive[j].T.fixed = False
        model.reactive[j].T.setub(t)
        model.reactive[j].T = t
    model.condenser.L['out'].fix(0)
    
    results_anchor = opt.solve(model,tee=False)
    model.solutions.store_to(results_anchor)
    
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.condenser.L['out'] - sum(model.reactive[j].MPCC.pf for j in model.TRAY),sense=pe.maximize)
    model.condenser.L['out'].fixed = False
    for j in model.reactive:
        model.reactive[j].T.fixed=True
    
    print('\nWorking on T = {} K'.format(model.reactive[1].T.value))
    print('-'*108)
    
    cd_data = {};
    cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
    cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {}; cd_data['b'] = {}
    for i in m.COMP_TOTAL:
        cd_data['x'][i] = []
        cd_data['y'][i] = []
        cd_data['g'][i] = []
        cd_data['d'][i] = []
        cd_data['b'][i] = []

    rf_data = {}
    for j in model.reactive:
        rf_data[j] = {}
        rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
        rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
        rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []; rf_data[j]['pf'] = []
        for i in m.COMP_TOTAL:
            rf_data[j]['r'][i] = []
            rf_data[j]['b'][i] = []
            rf_data[j]['x'][i] = []
            rf_data[j]['y'][i] = []       

    for re in Refluxrange:
        model.condenser.L['out'].setub(re)
        results = opt.solve(model,tee=False)
        update_dual(pe,model)
        print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
              format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reactive[3].L['out'].value))

        cd_data['V'].append(model.condenser.V['out'].value)
        cd_data['D'].append(model.condenser.L['P'].value)
        cd_data['Re'].append(model.condenser.L['out'].value)

        for i in model.reactive[1].r_total_comp:
            cd_data['x'][i].append(model.condenser.x[i].value)
            cd_data['y'][i].append(model.condenser.y[i].value)
            cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
            cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
            cd_data['b'][i].append(model.condenser.x[i].value*model.condenser.L['out'].value)

        for j in model.reactive:      
            rf_data[j]['T'].append(model.reactive[j].T.value)
            rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
            rf_data[j]['V'].append(model.reactive[j].V['out'].value)
            rf_data[j]['L'].append(model.reactive[j].L['out'].value)
            rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
            rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)
            rf_data[j]['pf'].append(model.reactive[j].MPCC.pf.value)

            for i in model.reactive[1].r_total_comp:
                rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
                rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
                rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
                rf_data[j]['y'][i].append(model.reactive[j].y[i].value)
        
    cd_data_master[t] = cd_data
    rf_data_master[t] = rf_data

In [ ]:
cnumber_range = range(1,57)

In [ ]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [ ]:
g_data = {}
d_data = {}
lr_data = {}
l1_data = {}
l2_data = {}
l3_data = {}

cd_x_data = {}
rf_x_data = {}

for t in Trange:

    g_data[t] = trans_cnumber(cd_data_master[t]['g'])
    d_data[t] = trans_cnumber(cd_data_master[t]['d'])
    lr_data[t] = trans_cnumber(cd_data_master[t]['d'])
    l1_data[t] = trans_cnumber(rf_data_master[t][1]['b'])
    l2_data[t] = trans_cnumber(rf_data_master[t][2]['b'])
    l3_data[t] = trans_cnumber(rf_data_master[t][3]['b'])
    
    cd_x_data[t] = trans_cnumber(cd_data_master[t]['x'])
    rf_x_data[t] = {}
    for j in model.reactive:
        rf_x_data[t][j] = trans_cnumber(rf_data_master[t][j]['x'])

In [ ]:
 def plot_distribution(index,temperature):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[temperature][index],'co-')
    ax.plot(cnumber_range,d_data[temperature][index],'go-')
    ax.plot(cnumber_range,lr_data[temperature][index],'ro-',alpha=0.1)
    ax.plot(cnumber_range,l1_data[temperature][index],'ro-',alpha=0.3)
    ax.plot(cnumber_range,l2_data[temperature][index],'ro-',alpha=0.5)
    ax.plot(cnumber_range,l3_data[temperature][index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Reflux','L1','L2','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data_master[temperature]['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[temperature][index],'go-')
    ax2.plot(cnumber_range,rf_x_data[temperature][1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[temperature][2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[temperature][3][index],'ro-')

    ax2.set_ylim(0, 0.2)
    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [ ]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:'),temperature=widgets.SelectionSlider(
    options=Trange,
    value=Trange[0],
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)
);

In [ ]:
plot_distribution(20,523.15)